# Guacamoleの起動

---

Guacamole環境を起動します。

## 全体構成

![構成](images/guacamole-201-01.png)

# 準備

![準備](images/guacamole-002-02.png)

## UnitGroup名の指定

構築する UnitGroup の名前を指定してください。

In [1]:
unit_group = 'Guacamole-C'

次のセルを実行すると「01-パラメータの設定.ipynb」で指定したパラメータを読み込みます。読み込むパラメータの値は、上のセルで指定した UnitGroup名に対応するものになります。UnitGroup名の指定が誤っていると意図したパラメータが読み込めないので注意してください。

In [2]:
%run scripts/group.py
gvars = load_group_vars(unit_group)

>「01-パラメータの設定.ipynb」で group_vars ファイルに保存した値を読み込むために、事前に作成した Python のスクリプト `scripts/group.py` を利用しています。このスクリプトでは、指定された UnitGroup名に対応する YAMLファイルを `group_vars/` ディレクトリから読み込み、その値を Python の辞書型変数に格納しています。

## VCCアクセストークンの入力

VCCにアクセスするためのトークンを入力します。

> VCCのアクセストークンは秘密情報となるためNotebook環境には保存してありません。他の秘密情報については VCC の Vaultサーバに保存してあります。しかしVCCのVaultサーバにアクセスするためにはVCCのアクセストークンが必要となります。そのため、Notebookを新たに実行する度にVCCのアクセストークンを入力する必要があります。

次のセルを実行すると入力枠が表示されるのでアクセストークンの値を入力してください。

> アクセストークン入力後に Enter キーを押すことで入力が完了します。

In [3]:
from getpass import getpass
vcc_access_token = getpass()

········


入力されたアクセストークンが正しいことを、実際にVCCにアクセスして確認します。

In [4]:
from common import logsetting
from vcpsdk.vcpsdk import VcpSDK

vcp = VcpSDK(vcc_access_token)

上のセルの実行結果がエラーとなり以下のようなメッセージが表示されている場合は、入力されたアクセストークンに誤りがあります。

```
2018-09-XX XX:XX:XX,XXX - ERROR - config vc failed: http_status(403)
2018-09-XX XX:XX:XX,XXX - ERROR - 2018/XX/XX XX:XX:XX UTC: VCPAuthException: xxxxxxx:token lookup is failed: permission denied
```

エラーになった場合はこの節のセルを全て `unfreeze` してから、もう一度アクセストークンの入力を行ってください。

## AnsibleからVaultサーバにアクセスするための準備

このNotebookでは、VCノードに環境を構築するために Ansible を用います。「01-パラメータの設定.ipynb」で group_varsファイルに記録したパラメータはAnsibleから直接参照することができます。しかしVCCのVaultサーバに記録したパラメータは、そのままではAnsibleから参照することができません。そこで Ansible から Vaultサーバの値を参照するために必要となるライブラリをインストールします。また VCC の Vaultサーバにアクセスするために必要となるトークンと Vault サーバのアドレスを環境変数に設定します。

まず Vaultサーバにアクセスするために必要となるライブラリのインストールを行います。

In [5]:
!pip2 install hvac --user

You are using pip version 18.0, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


VCCのVault サーバにアクセスする際に必要となるパラメータを環境変数に設定します。

In [6]:
import os
os.environ['VAULT_ADDR'] = vcp.vcc_info()['vault_url']
os.environ['VAULT_TOKEN'] = vcc_access_token

# VCノードの起動

VCP SDK を利用してVCノードの起動を行います。

![VCノードの起動](images/guacamole-002-03.png)

## Guacamole用VCノードの spec を指定する

「01-パラメータの設定.ipynb」で指定したパラメータをGuacamole用VCノードの `spec` に設定します。

`spec`に設定するパラメータを以下に示します。

* `image`: Baseコンテナイメージ
  - Guacamole用 Baseコンテナイメージを設定します
* `params_v`: ボリューム設定
  - Baseコンテナのボリュームを設定します
  - Guacamole用Baseコンテナではホスト側の `/sys/fs/cgroup` をコンテナから見えるように設定する必要があります
* `ip_addresses`: IPアドレス
  - VCノードに割り当てるプライベートIPアドレスを設定します
* `volume_size`: ルートボリュームサイズ
  - VCノードに割り当てるルートボリュームサイズ(GB)
* `set_ssh_publickey()`: SSHの公開鍵
  - VCノードに登録するSSHの公開鍵
* `params_e`: Baseコンテナの環境変数設定
  - `USER_CONTAINER_IMAGE`
     - Baseコンテナ起動時に、自動的に取得を行うアプリケーションコンテナイメージのリスト

In [7]:
spec_srv = vcp.get_spec(gvars['vc_provider'], gvars['vc_flavor'])

# Guacamole用の Baseコンテナイメージを指定する
spec_srv.image = 'vcp/guacamole:1.4.2-server'
spec_srv.params_v = ['/sys/fs/cgroup:/sys/fs/cgroup:ro']

# ルートボリュームサイズを指定する
spec_srv.volume_size = gvars['vc_disk_size']

# Guacamole用VCノードに割り当てるIPアドレスを設定する
spec_srv.ip_addresses = [gvars['vc_guacamole_ipaddress']]

# VCノードにsshでログインするための公開鍵を設定する
spec_srv.set_ssh_pubkey(gvars['ssh_public_key_path'])

# 起動時に取得するイメージ名を選択する
# ハンズオンの時間短縮用
container_images = [
    'guacamole/guacamole:0.9.14',
    'guacamole/guacd:0.9.14',
    'mariadb:10.3',
    'nginx:1.14',
    'prom/prometheus',
]
spec_srv.params_e.append('USER_CONTAINER_IMAGE=' + ','.join(container_images))

`spec` の設定値を確認します。

In [8]:
print(spec_srv.cci(''))


- unit:
    cci_version: "1.0"
    name: 
    description: ""
    private_network: default
    num_nodes: 1
    image: vcp/guacamole:1.4.2-server
    recovery_mode: none
    params:
        v:
        - /sys/fs/cgroup:/sys/fs/cgroup:ro
        e:
        - AUTHORIZED_KEYS=c3NoLXJzYSBBQUFBQjNOemFDMXljMkVBQUFBREFRQUJBQUFCQVFDcW5XY3Y2TlB6SVRld3pKdEMzbTdPb040MGhHUkJQSDdRR2pjUk5FQXY5T0tMcFl6TEJha01rN2VLL3krRzZZRTQ3VUJCdWFGT21hREtOdVJsdHRjLytZZWRTbEI3eGdtMXpyUXl4SExpa2FxSys3azBxclRkSHNPcm5pNEJOWDNEWS9IK2pINldGZ3RZSUtxUk1qWmRXM05PRWJabHMwZkNBVFovR3R6RkhnblMrKzhXN3BSUXYzc1R4NDBHR2h0L2dWU2hOcGEyRmtKREFYeFFscDZXK1AvdkNBaDJxV3E5M1ZRQ1FzQWxpNjVCTGNMZ0hPVy9UbGFlZ2pTTXk4dkszNzUrL0NuSTZja1BTeU1pWHlvMi91clRneXg5QlNZQWYvVUovNXpnb3hjSUsxUlFqR1FPcUs2MVZUTVo2QnhzWXc0bkRDaWdwaFI5L1BXMnNIUDMgYml0X2t1bkA5M2EzNGM3NmQwNDIK
        - USER_CONTAINER_IMAGE=guacamole/guacamole:0.9.14,guacamole/guacd:0.9.14,mariadb:10.3,nginx:1.14,prom/prometheus
    cloud_provider: aws
    cloud_params:
        cci_version: "1.1"

## 演習環境用VCノードのspec を指定する

「01-パラメータの設定.ipynb」で指定したパラメータを演習環境用VCノードの `spec` に設定します。

`spec`に設定するパラメータを以下に示します。

* `image`: Baseコンテナイメージ
  - 演習環境用 Baseコンテナイメージを設定します
* `params_v`: ボリューム設定
  - Baseコンテナのボリュームを設定します
  - 演習環境用Baseコンテナではホスト側の `/sys/fs/cgroup` をコンテナから見えるように設定する必要があります
* `volume_size`: ルートボリュームサイズ
  - VCノードに割り当てるルートボリュームサイズ(GB)
* `set_ssh_publickey()`: SSHの公開鍵
  - VCノードに登録するSSHの公開鍵
* `params_e`: Baseコンテナの環境変数設定
  - `MASTER_ADDRESS`
     - Guacamole用VCノードに割り当てるIPアドレス
  - `NFS_DOMAIN`
     - NFSのドメイン名
  - `ETC_FSTAB`
     - Baseコンテナに設定する `/etc/fstab` を base64エンコードした値
  - `MKDIR_LIST`
     - 起動時に作成するディレクトリ
  - `USER_CONTAINER_IMAGE`
     - Baseコンテナ起動時に、自動的に取得を行うアプリケーションコンテナイメージのリスト

In [9]:
from base64 import b64encode

spec_cli = vcp.get_spec(gvars['vc_provider'], gvars['vc_flavor'])

# 演習環境用の Baseコンテナイメージを指定する
spec_cli.image = 'vcp/guacamole:1.4.2-client'
spec_cli.params_v = ['/sys/fs/cgroup:/sys/fs/cgroup:ro']

# ルートボリュームサイズを指定する
spec_cli.volume_size = gvars['vc_disk_size']

# NFSサーバのホームディレクトリをマウントする /etc/fstab のエントリ
etc_fstab = f'{gvars["nfs_address"]}:{gvars["nfs_directory"]}  /alt/home nfs {gvars["nfs_mount_options"]} 0 0'

# 起動時に取得するイメージ名を選択する
if 'user_container_image_base' in gvars:
    # 演習環境コンテナをカスタマイズする場合は、まだイメージが存在していないので元になるイメージを取得する
    container_image = gvars['user_container_image_base']
else:
    # 演習環境コンテナをカスタマイズしない場合は、演習環境コンテナイメージを取得する
    container_image = gvars['user_container_image']

spec_cli.params_e = [
    # Guacamoleを実行するVCノードのIPアドレスを設定する
    f"MASTER_ADDRESS={gvars['vc_guacamole_ipaddress']}",
    # NFSのドメイン名を設定する
    f"NFS_DOMAIN={gvars['nfs_domain']}",
    # /etc/fstab に追加するエントリを設定する
    f"ETC_FSTAB={b64encode(etc_fstab.encode('UTF-8')).decode('UTF-8')}",
    # NFSでマウントするマウントポイントのディレクトリを設定する
    "MKDIR_LIST=/alt/home",
    # 起動時に取得するコンテナイメージ名を設定する
    f"USER_CONTAINER_IMAGE=centos:7,{container_image}",
]

# VCノードにsshでログインするための公開鍵を設定する
spec_cli.set_ssh_pubkey(gvars['ssh_public_key_path'])

`spec` の設定値を確認します。

In [10]:
print(spec_cli.cci(''))


- unit:
    cci_version: "1.0"
    name: 
    description: ""
    private_network: default
    num_nodes: 1
    image: vcp/guacamole:1.4.2-client
    recovery_mode: none
    params:
        v:
        - /sys/fs/cgroup:/sys/fs/cgroup:ro
        e:
        - MASTER_ADDRESS=172.30.2.100
        - NFS_DOMAIN=vcp-handson.org
        - ETC_FSTAB=MTcyLjMwLjIuNTA6LyAgL2FsdC9ob21lIG5mcyBuZnN2ZXJzPTQuMiAwIDA=
        - MKDIR_LIST=/alt/home
        - USER_CONTAINER_IMAGE=centos:7,192.168.2.1:5001/vcp/xrdp:rstudio
        - AUTHORIZED_KEYS=c3NoLXJzYSBBQUFBQjNOemFDMXljMkVBQUFBREFRQUJBQUFCQVFDcW5XY3Y2TlB6SVRld3pKdEMzbTdPb040MGhHUkJQSDdRR2pjUk5FQXY5T0tMcFl6TEJha01rN2VLL3krRzZZRTQ3VUJCdWFGT21hREtOdVJsdHRjLytZZWRTbEI3eGdtMXpyUXl4SExpa2FxSys3azBxclRkSHNPcm5pNEJOWDNEWS9IK2pINldGZ3RZSUtxUk1qWmRXM05PRWJabHMwZkNBVFovR3R6RkhnblMrKzhXN3BSUXYzc1R4NDBHR2h0L2dWU2hOcGEyRmtKREFYeFFscDZXK1AvdkNBaDJxV3E5M1ZRQ1FzQWxpNjVCTGNMZ0hPVy9UbGFlZ2pTTXk4dkszNzUrL0NuSTZja1BTeU1pWHlvMi91clRneXg5QlNZQWYvVUovNXpnb3hjSUsxUlFqR1FPc

## VCノードの起動

UnitGroupを作成します。

In [11]:
ugroup = vcp.create_ugroup(unit_group)

VCノードを起動します。

> VCノードの起動には４分程度かかります。

In [12]:
unit_srv = ugroup.create_unit('server', spec_srv)
unit_cli = ugroup.create_unit('client', spec_cli)

2019-03-20 19:50:59,521 - INFO - BOOTING ... 0 sec
2019-03-20 19:51:04,719 - INFO - BOOTING ... 5 sec
2019-03-20 19:51:09,915 - INFO - BOOTING ... 10 sec
2019-03-20 19:51:15,117 - INFO - BOOTING ... 15 sec
2019-03-20 19:51:20,319 - INFO - BOOTING ... 20 sec
2019-03-20 19:51:25,519 - INFO - BOOTING ... 25 sec
2019-03-20 19:51:30,726 - INFO - BOOTING ... 30 sec
2019-03-20 19:51:35,925 - INFO - BOOTING ... 35 sec
2019-03-20 19:51:41,131 - INFO - BOOTING ... 40 sec
2019-03-20 19:51:46,342 - INFO - BOOTING ... 45 sec
2019-03-20 19:51:51,769 - INFO - BOOTING ... 50 sec
2019-03-20 19:51:57,010 - INFO - BOOTING ... 55 sec
2019-03-20 19:52:02,260 - INFO - BOOTING ... 60 sec
2019-03-20 19:52:07,485 - INFO - BOOTING ... 65 sec
2019-03-20 19:52:12,691 - INFO - BOOTING ... 70 sec
2019-03-20 19:52:17,893 - INFO - BOOTING ... 75 sec
2019-03-20 19:52:23,092 - INFO - BOOTING ... 80 sec
2019-03-20 19:52:28,299 - INFO - BOOTING ... 85 sec
2019-03-20 19:52:33,503 - INFO - BOOTING ... 90 sec
2019-03-20 19:

起動したVCノードの一覧を表示します。

> VCノードの状態が `ERROR` と表示されている場合は VCノードの起動に失敗しています。`ugroup.cleanup()` を実行してエラーとなった VCノードを削除してください。

In [13]:
ugroup.df_nodes()

,vcno,vcname,unit_name,unit_state,node_no,node_id,node_state,cloud_instance_address,cloud_instance_id,cloud_instance_name,volumes
0,49,Guacamole-C,server,RUNNING,1,b09079c8...,RUNNING,172.30.2.100,i-04c09d934a6133a83,VCP-e84fbb63-88b49866,none
1,49,Guacamole-C,client,RUNNING,1,bb3fa509...,RUNNING,172.30.2.167,i-0a0c61fa15a65c6e6,VCP-e84fbb63-88b49866,none


# Ansibleの設定

VCノードをAnsibleで操作するための設定を行います。

![Ansibleの設定](images/guacamole-002-04.png)

まず、VCノードにSSHでログインできるようにするために `~/.ssh/known_hosts` の更新を行います。

> 何度かVCノードの起動を行うと、異なるホストが同じIPアドレスで起動するためにSSHのホストキーのチェックでエラーになる事があります。このような状況に対応するために、起動したVCノードのIPアドレスに対応するエントリを`known_hosts`ファイルから削除します。その後、`ssh-keyscan`コマンドを利用して起動したVCノードのホストキーを取得して `known_hosts`ファイルの内容を更新します。

In [14]:
from time import sleep

def check_update_known_hosts(ipaddr):
    # VCノード起動直後だと sshd サービスが開始されておらずに known_hosts が更新されない場合がある
    # ssh-keyscan が値を取得できるまで何度かリトライする
    for x in range(10):
        out = ! echo $(ssh-keyscan {ipaddr} 2> /dev/null | wc -l)
        update_lines = int(out[0])
        if update_lines > 0:
            break
        sleep(1)
    else:
        raise RuntimeError("ERROR: timeout!")    

for addr in ugroup.find_ip_addresses():
    !ssh-keygen -R {addr}
    check_update_known_hosts(addr)
    !ssh-keyscan -H {addr} >> ~/.ssh/known_hosts

# Host 172.30.2.100 found: line 37 type ED25519
# Host 172.30.2.100 found: line 38 type RSA
# Host 172.30.2.100 found: line 39 type ECDSA
/home/bit_kun/.ssh/known_hosts updated.
Original contents retained as /home/bit_kun/.ssh/known_hosts.old
# 172.30.2.100 SSH-2.0-OpenSSH_7.4
# 172.30.2.100 SSH-2.0-OpenSSH_7.4
# 172.30.2.100 SSH-2.0-OpenSSH_7.4
/home/bit_kun/.ssh/known_hosts updated.
Original contents retained as /home/bit_kun/.ssh/known_hosts.old
# 172.30.2.167 SSH-2.0-OpenSSH_7.4
# 172.30.2.167 SSH-2.0-OpenSSH_7.4
# 172.30.2.167 SSH-2.0-OpenSSH_7.4


起動したVCノードに対応するエントリを Ansible のインベントリに登録します。

> Ansibleで操作を行うためには、操作対象のホスト(IPアドレス)をインベントリに登録する必要があります。

In [15]:
import os

with open('hosts', 'w') as f:
    f.write(f'''
[{unit_group}:children]
{unit_group}-server
{unit_group}-client
    
[{unit_group}-server]
{os.linesep.join(unit_srv.find_ip_addresses())}

[{unit_group}-client]
{os.linesep.join(unit_cli.find_ip_addresses())}

[{unit_group}:vars]
ansible_ssh_private_key_file={gvars["ssh_private_key_path"]}
''')
    
!cat ./hosts


[Guacamole-C:children]
Guacamole-C-server
Guacamole-C-client
    
[Guacamole-C-server]
172.30.2.100

[Guacamole-C-client]
172.30.2.167

[Guacamole-C:vars]
ansible_ssh_private_key_file=/home/bit_kun/.ssh/id_rsa


複数のVCノードをまとめて操作しやすくするためにインベントリには Ansibleのグループを定義しておきます。UnitGroupに属する全てのVCノードが所属するAnsible のグループとして UnitGroup名と同じAnsibleのグループを定義します。ここでは `Guacamole-C`というAnsibleのグループを定義して、そこに全てのVCノードのIPアドレスを登録してあります。また、Guacamole用Unitと演習環境用Unitに対応するAnsibleのグループも定義しておきます。

先程VCノードを登録したファイルをインベントリとして指定するためのAnsibleのコンフィギュレーションファイルを作成します。

> カレントディレクトリにコンフィギュレーションファイル(`ansible.cfg`)を作成すると、Ansibleを実行する際にその設定が適用されます。

In [16]:
from pathlib import Path

inventory = Path('./hosts')
with open('ansible.cfg', 'w') as f:
    f.write(f'''
[defaults]
inventory = {inventory.absolute()}
''')
    
!cat ./ansible.cfg


[defaults]
inventory = /notebooks/notebook/handson/guacamole/hosts


UnitGroupに属する全てのVCノードに対して Ansible で接続できることを確認します。

> ここでは、複数のVCノードをまとめて扱うためにAnsibleのグループを指定しています。グループ名は UnitGroup名と同じ値にしてあります。

In [17]:
!ansible {unit_group} -m ping

172.30.2.100 | SUCCESS => {
    "changed": false, 
    "ping": "pong"
}
172.30.2.167 | SUCCESS => {
    "changed": false, 
    "ping": "pong"
}


# Guacamole環境の設定ファイルの配置

![Guacamole環境の設定ファイルの配置](images/guacamole-002-05.png)

## コンテナ構成の設定ファイルを配置する

Guacamole環境を構成するコンテナ群を定義するための設定ファイル（`docker-compose.yml`）などをGuacamole用VCノードに配置します。

> `docker-compose`コマンドを利用すると`docker-compose.yml`ファイルに記述された内容に従い複数のコンテナの起動、停止などの操作を行うことができます。

In [18]:
# 配置先のディレクトリの作成
!ansible {unit_group}-server -b -m file -a 'path=/opt/guacamole \
    state=directory owner=vcp group=vcp'
!ansible {unit_group}-server -b -m file -a 'path=/opt/guacamole/sql \
    state=directory owner=vcp group=vcp'
!ansible {unit_group}-server -b -m file -a 'path=/opt/guacamole/ldap \
    state=directory owner=vcp group=vcp'
# 設定ファイルの配置
!ansible {unit_group}-server -m synchronize \
    -a 'src=template/docker/compose/ dest=/opt/guacamole'
!ansible {unit_group}-server -m template \
    -a 'src=template/docker/compose/docker-compose.yml dest=/opt/guacamole/'
!ansible {unit_group}-server -m template \
    -a 'src=template/docker/compose/prometheus.yml dest=/opt/guacamole/'

172.30.2.100 | SUCCESS => {
    "changed": true, 
    "gid": 1000, 
    "group": "vcp", 
    "mode": "0755", 
    "owner": "vcp", 
    "path": "/opt/guacamole", 
    "size": 4096, 
    "state": "directory", 
    "uid": 1000
}
172.30.2.100 | SUCCESS => {
    "changed": true, 
    "gid": 1000, 
    "group": "vcp", 
    "mode": "0755", 
    "owner": "vcp", 
    "path": "/opt/guacamole/sql", 
    "size": 4096, 
    "state": "directory", 
    "uid": 1000
}
172.30.2.100 | SUCCESS => {
    "changed": true, 
    "gid": 1000, 
    "group": "vcp", 
    "mode": "0755", 
    "owner": "vcp", 
    "path": "/opt/guacamole/ldap", 
    "size": 4096, 
    "state": "directory", 
    "uid": 1000
}
172.30.2.100 | SUCCESS => {
    "changed": true, 
    "cmd": "/usr/bin/rsync --delay-updates -F --compress --archive --rsh=/usr/bin/ssh -S none -i /home/bit_kun/.ssh/id_rsa -o StrictHostKeyChecking=no -o UserKnownHostsFile=/dev/null --out-format=<<CHANGED>>%i %n%L /notebooks/notebook/handson/guacamole/template/d

## Guacamoleの初期設定SQLファイルの作成

Guacamole用のテーブル作成などを行う初期実行SQLファイルを作成します。

Guacamoleのコンテナイメージに含まれている`initdb.sh` スクリプトを実行するとGuacamoleが利用するデータベースを作成するためのSQLを生成することができます。ここではMariaDB(MySQL)用のデータベース作成SQLの生成を行います。

In [19]:
!ansible {unit_group}-server -m file -a 'path=/opt/guacamole/sql/initdb.sql state=touch'
!ansible {unit_group}-server -a 'chdir=/opt/guacamole docker-compose -f docker-compose-init.yml up -d'
!ansible {unit_group}-server -a 'chdir=/opt/guacamole docker-compose -f docker-compose-init.yml down'

172.30.2.100 | SUCCESS => {
    "changed": true, 
    "dest": "/opt/guacamole/sql/initdb.sql", 
    "gid": 1000, 
    "group": "vcp", 
    "mode": "0664", 
    "owner": "vcp", 
    "size": 0, 
    "state": "file", 
    "uid": 1000
}
172.30.2.100 | SUCCESS | rc=0 >>
Creating network "guacamole_default" with the default driver
32mCreating guacamole_guacamole-initdb_1 ... 
 guacamole_guacamole-initdb_1 ... done

172.30.2.100 | SUCCESS | rc=0 >>
32mRemoving guacamole_guacamole-initdb_1 ... 
Removing network guacamole_defaultdone



SQLファイルが作成されたことを確認します。

In [20]:
!ansible {unit_group}-server -a 'head -30 /opt/guacamole/sql/initdb.sql'

172.30.2.100 | SUCCESS | rc=0 >>
--
-- Licensed to the Apache Software Foundation (ASF) under one
-- or more contributor license agreements.  See the NOTICE file
-- distributed with this work for additional information
-- regarding copyright ownership.  The ASF licenses this file
-- to you under the Apache License, Version 2.0 (the
-- "License"); you may not use this file except in compliance
-- with the License.  You may obtain a copy of the License at
--
--   http://www.apache.org/licenses/LICENSE-2.0
--
-- Unless required by applicable law or agreed to in writing,
-- software distributed under the License is distributed on an
-- "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY
-- KIND, either express or implied.  See the License for the
-- specific language governing permissions and limitations
-- under the License.
--

--
-- Table of connection groups. Each connection group has a name.
--

CREATE TABLE `guacamole_connection_group` (

  `connection_group_id`   int(11)      NOT

## LDAPサーバのCA証明書の配置

LDAPサーバにSSL/TSLで接続する際に必要となるCA証明書をVCノードに配置します。

In [21]:
if 'ldap_ca_path' in gvars:
    !ansible {unit_group}-server -m copy \
        -a 'dest=/opt/guacamole/ldap/ldap-ca.crt src={{{{ldap_ca_path}}}}'

172.30.2.100 | SUCCESS => {
    "changed": true, 
    "checksum": "d5b33cff2ebb27106ef7dafd467af3165ae43927", 
    "dest": "/opt/guacamole/ldap/ldap-ca.crt", 
    "gid": 1000, 
    "group": "vcp", 
    "md5sum": "21ff606330640a3d2b75aa6d5b83a65a", 
    "mode": "0664", 
    "owner": "vcp", 
    "size": 1042, 
    "src": "/home/vcp/.ansible/tmp/ansible-tmp-1553079330.06-7974914317083/source", 
    "state": "file", 
    "uid": 1000
}


証明書ファイルが配置されたことを確認します。

In [22]:
if 'ldap_ca_path' in gvars:
    !ansible {unit_group}-server -a 'cat /opt/guacamole/ldap/ldap-ca.crt'

172.30.2.100 | SUCCESS | rc=0 >>
-----BEGIN CERTIFICATE-----
MIIC0zCCAlmgAwIBAgIUCfQ+m0pgZ/BjYAJvxrn/bdGNZokwCgYIKoZIzj0EAwMw
gZYxCzAJBgNVBAYTAlVTMRUwEwYDVQQKEwxBMUEgQ2FyIFdhc2gxJDAiBgNVBAsT
G0luZm9ybWF0aW9uIFRlY2hub2xvZ3kgRGVwLjEUMBIGA1UEBxMLQWxidXF1ZXJx
dWUxEzARBgNVBAgTCk5ldyBNZXhpY28xHzAdBgNVBAMTFmRvY2tlci1saWdodC1i
YXNlaW1hZ2UwHhcNMTUxMjIzMTM1MzAwWhcNMjAxMjIxMTM1MzAwWjCBljELMAkG
A1UEBhMCVVMxFTATBgNVBAoTDEExQSBDYXIgV2FzaDEkMCIGA1UECxMbSW5mb3Jt
YXRpb24gVGVjaG5vbG9neSBEZXAuMRQwEgYDVQQHEwtBbGJ1cXVlcnF1ZTETMBEG
A1UECBMKTmV3IE1leGljbzEfMB0GA1UEAxMWZG9ja2VyLWxpZ2h0LWJhc2VpbWFn
ZTB2MBAGByqGSM49AgEGBSuBBAAiA2IABMZf/12pupAgl8Sm+j8GmjNeNbSFAZWW
oTmIvf2Mu4LWPHy4bTldkQgHUbBpT3xWz8f0lB/ru7596CHsGoL2A28hxuclq5hb
Ux1yrIt3bJIY3TuiX25HGTe6kGCJPB1aLaNmMGQwDgYDVR0PAQH/BAQDAgEGMBIG
A1UdEwEB/wQIMAYBAf8CAQIwHQYDVR0OBBYEFE+l6XolXDAYnGLTl4W6ULKHrm74
MB8GA1UdIwQYMBaAFE+l6XolXDAYnGLTl4W6ULKHrm74MAoGCCqGSM49BAMDA2gA
MGUCMQCXLZj8okyxW6UTL7hribUUbu63PbjuwIXnwi420DdNsvA9A7fcQEXScWFL
XAGC8rkCMGcqwXZPSRfwuI9r+R11g

## 演習環境コンテナのパラメータファイル

演習環境コンテナを起動する際に必要となるパラメータを Guacamole用 VCノードの `/etc/sysconfig/guacamole-proxy` に記述しておきます。

In [23]:
!ansible {unit_group}-server -m template \
    -b -a 'dest=/etc/sysconfig/guacamole-proxy \
        src=template/sysconfig/guacamole-proxy mode=0600'

172.30.2.100 | SUCCESS => {
    "changed": true, 
    "checksum": "b511cce0d62acdff8b671db0c397f00cd71f1672", 
    "dest": "/etc/sysconfig/guacamole-proxy", 
    "gid": 0, 
    "group": "root", 
    "md5sum": "aebc897a95ce67a75fad5271deec818f", 
    "mode": "0600", 
    "owner": "root", 
    "size": 596, 
    "src": "/home/vcp/.ansible/tmp/ansible-tmp-1553079335.08-212809797324777/source", 
    "state": "file", 
    "uid": 0
}


設定ファイルが配置されたことを確認します。

In [24]:
!ansible {unit_group}-server -a 'ls -la /etc/sysconfig/guacamole-proxy'

172.30.2.100 | SUCCESS | rc=0 >>
-rw------- 1 root root 596 Mar 20 10:55 /etc/sysconfig/guacamole-proxy



## リバースプロキシの設定ファイルの配置

外部からのアクセスを SSL/TLS にするためにひつようとなる、サーバ証明書と秘密鍵ファイルをVCノードに配置します。配置した証明書ファイルなどはリバースプロキシとなる nginx コンテナに読み込まれます。

In [25]:
# 配置先のディレクトリの作成
!ansible {unit_group}-server -b -m file -a 'path=/opt/guacamole/nginx \
    state=directory owner=vcp group=vcp'
!ansible {unit_group}-server -b -m file -a 'path=/opt/guacamole/nginx/certs \
    state=directory owner=vcp group=vcp'
# 設定ファイルの配置
!ansible {unit_group}-server -m template \
    -a 'src={{{{nginx_conf}}}} dest=/opt/guacamole/nginx/default.conf'
!ansible {unit_group}-server -m copy \
    -a 'src={{{{nginx_tls_cert}}}} dest=/opt/guacamole/nginx/certs/server.crt'
!ansible {unit_group}-server -m copy \
    -a 'src={{{{nginx_tls_key}}}} dest=/opt/guacamole/nginx/certs/server.key'

172.30.2.100 | SUCCESS => {
    "changed": true, 
    "gid": 1000, 
    "group": "vcp", 
    "mode": "0755", 
    "owner": "vcp", 
    "path": "/opt/guacamole/nginx", 
    "size": 4096, 
    "state": "directory", 
    "uid": 1000
}
172.30.2.100 | SUCCESS => {
    "changed": true, 
    "gid": 1000, 
    "group": "vcp", 
    "mode": "0755", 
    "owner": "vcp", 
    "path": "/opt/guacamole/nginx/certs", 
    "size": 4096, 
    "state": "directory", 
    "uid": 1000
}
172.30.2.100 | SUCCESS => {
    "changed": true, 
    "checksum": "3abefbf610a4799b969d1f5e738d666a5894fc56", 
    "dest": "/opt/guacamole/nginx/default.conf", 
    "gid": 1000, 
    "group": "vcp", 
    "md5sum": "4a99280299e4df73f5f9163ff3f9a52a", 
    "mode": "0664", 
    "owner": "vcp", 
    "size": 622, 
    "src": "/home/vcp/.ansible/tmp/ansible-tmp-1553079343.06-135023020918387/source", 
    "state": "file", 
    "uid": 1000
}
172.30.2.100 | SUCCESS => {
    "changed": true, 
    "checksum": "2acdd580ad79346e27c7b7981

# Guacamoleの起動

Guacamole環境を構成するコンテナ群を起動します。

![Guacamoleの起動](images/guacamole-002-06.png)

コンテナ構成は`docker-compose.yml`ファイルに記述してあります。そのため、コンテナ群の起動などの操作は`docker-compose`コマンドを用いて行います。

まず、Guacamole環境を構成するコンテナ群のイメージを取得します。

In [26]:
!ansible {unit_group}-server -a 'chdir=/opt/guacamole docker-compose pull'

172.30.2.100 | SUCCESS | rc=0 >>









コンテナを起動します。

In [27]:
!ansible {unit_group}-server -a 'chdir=/opt/guacamole docker-compose up -d'

172.30.2.100 | SUCCESS | rc=0 >>
Creating network "guacamole_default" with the default driver









起動したコンテナの状態を確認します。コンテナが正常に起動していれば`State`が`Up`と表示されます。全てのコンテナの状態が `Up`となっていることを確認してください。

起動に成功した場合の出力結果の例を以下に示します。
```
172.30.2.100 | SUCCESS | rc=0 >>
         Name                       Command               State              Ports            
----------------------------------------------------------------------------------------------
guacamole_database_1     docker-entrypoint.sh mysqld      Up      0.0.0.0:3306->3306/tcp      
guacamole_guacamole_1    /tmp/init.sh                     Up      8080/tcp                    
guacamole_guacd_1        /usr/local/sbin/guacd -b 0 ...   Up      4822/tcp                    
guacamole_nginx_1        nginx -g daemon off;             Up      0.0.0.0:443->443/tcp, 80/tcp
guacamole_prometheus_1   /bin/prometheus --config.f ...   Up      0.0.0.0:9090->9090/tcp      
```

In [28]:
!ansible {unit_group}-server -a 'chdir=/opt/guacamole docker-compose ps'

172.30.2.100 | SUCCESS | rc=0 >>
         Name                       Command               State              Ports            
----------------------------------------------------------------------------------------------
guacamole_database_1     docker-entrypoint.sh mysqld      Up      0.0.0.0:3306->3306/tcp      
guacamole_guacamole_1    /tmp/init.sh                     Up      8080/tcp                    
guacamole_guacd_1        /usr/local/sbin/guacd -b 0 ...   Up      4822/tcp                    
guacamole_nginx_1        nginx -g daemon off;             Up      0.0.0.0:443->443/tcp, 80/tcp
guacamole_prometheus_1   /bin/prometheus --config.f ...   Up      0.0.0.0:9090->9090/tcp      



5.2 節で作成したデータベースの初期化SQLの処理が完了して、Guacamole用のデータベースが作成するまでの待ち合わせを行います。次のセルを実行してください。

初期化処理が完了するまでは `ERROR` と表示されますが、最終的に処理が完了すると `SUCCESS` と表示されテーブルの一覧が表示されます。

表示例)
```
172.30.2.100 | FAILED | rc=1 >>
ERROR 1045 (28000): Access denied for user 'guacamole'@'localhost' (using password: YES)non-zero return code

172.30.2.100 | SUCCESS | rc=0 >>
Tables_in_guacamole
guacamole_connection
guacamole_connection_group
guacamole_connection_group_permission
guacamole_connection_history
guacamole_connection_parameter
guacamole_connection_permission
guacamole_sharing_profile
guacamole_sharing_profile_parameter
guacamole_sharing_profile_permission
guacamole_system_permission
guacamole_user
guacamole_user_history
guacamole_user_password_history
guacamole_user_permission
```

In [29]:
def show_guacamole_tables():
    !ansible {unit_group}-server -a \
        'chdir=/opt/guacamole docker-compose exec -T \
            -e MYSQL_PWD={{{{lookup("hashi_vault", "secret=" + guacamole_vault_path + ":db_password")}}}} \
            database mysql -u {{{{guacamole_db_user}}}} {{{{guacamole_db_name}}}} \
            -e "SHOW TABLES;"'

%run scripts/utils.py
retry_exec(show_guacamole_tables, interval=15, retry_max=20)

172.30.2.100 | FAILED | rc=1 >>
ERROR 1045 (28000): Access denied for user 'guacamole'@'localhost' (using password: YES)non-zero return code

172.30.2.100 | SUCCESS | rc=0 >>
Tables_in_guacamole
guacamole_connection
guacamole_connection_group
guacamole_connection_group_permission
guacamole_connection_history
guacamole_connection_parameter
guacamole_connection_permission
guacamole_sharing_profile
guacamole_sharing_profile_parameter
guacamole_sharing_profile_permission
guacamole_system_permission
guacamole_user
guacamole_user_history
guacamole_user_password_history
guacamole_user_permission



# Guacamoleの設定

起動した Guacamoleに対してユーザ登録と、管理者パスワードの変更を行います。

![Guacamoleの設定](images/guacamole-002-07.png)

## ユーザの登録

Guacamoleのユーザ登録を行います。

ここでは各ユーザ毎に１つの演習環境コンテナ割り当てる想定でユーザ登録を行います。

ユーザ登録では主に以下のような処理を行っています。
* データベースのGuacamoleユーザテーブルにユーザエントリを追加する
* データベースのGuacamoleコネクション設定テーブルに、ユーザ毎に作成する演習環境コンテナのコネクション(SSH, RDP)を追加する
* ユーザ毎の演習環境コンテナを定義する設定ファイルを作成する

ユーザ登録処理は、事前にスクリプトファイル`guacamole-update-users`に記述してあります。ここでは登録ユーザごとにスクリプトによる登録処理を実行します。

In [30]:
for user in gvars['guacamole_users']:
    !ansible {unit_group}-server -b -m shell \
        -a 'set -a; source /etc/sysconfig/guacamole-proxy; set +a; \
        /usr/local/bin/guacamole-update-users add {user}'

172.30.2.100 | SUCCESS | rc=0 >>


172.30.2.100 | SUCCESS | rc=0 >>


172.30.2.100 | SUCCESS | rc=0 >>


172.30.2.100 | SUCCESS | rc=0 >>


172.30.2.100 | SUCCESS | rc=0 >>


172.30.2.100 | SUCCESS | rc=0 >>


172.30.2.100 | SUCCESS | rc=0 >>


172.30.2.100 | SUCCESS | rc=0 >>


172.30.2.100 | SUCCESS | rc=0 >>


172.30.2.100 | SUCCESS | rc=0 >>




## 管理者パスワードの変更

Guacamoleの管理者パスワードはデフォルト値のままになっています。「01-パラメータの設定.ipynb」で指定した値に変更を行います。

管理者パスワードを更新するためのSQLファイルをVCノードに配置します。

In [31]:
!ansible {unit_group}-server -m template \
    -a 'src=template/sql/update-adminpw.sql dest=/opt/guacamole/sql/'

172.30.2.100 | SUCCESS => {
    "changed": true, 
    "checksum": "3961dc30168236bbd65ecf5d2e97484aef717aee", 
    "dest": "/opt/guacamole/sql/update-adminpw.sql", 
    "gid": 1000, 
    "group": "vcp", 
    "md5sum": "2917efe83244fce98f47fa5a20e66400", 
    "mode": "0664", 
    "owner": "vcp", 
    "size": 252, 
    "src": "/home/vcp/.ansible/tmp/ansible-tmp-1553079402.4-208978031164560/source", 
    "state": "file", 
    "uid": 1000
}


管理者パスワードを更新するSQLファイルを実行します。

In [32]:
!ansible {unit_group}-server -a \
    'chdir=/opt/guacamole docker-compose exec -T \
    -e MYSQL_PWD={{{{lookup("hashi_vault", "secret=" + guacamole_vault_path + ":db_password")}}}} \
    database mysql -u {{{{guacamole_db_user}}}} {{{{guacamole_db_name}}}} \
    -e "source /mnt/update-adminpw.sql"'

172.30.2.100 | SUCCESS | rc=0 >>




# 演習環境用コンテナイメージの配布

演習環境用のVCノードに演習環境コンテナイメージを配布します。

![演習環境用コンテナイメージの配布](images/guacamole-002-08.png)

## コンテナイメージの配布

演習環境用VCノードにコンテナイメージを配布します。　

In [33]:
!ansible {unit_group}-client -a 'docker pull {{{{user_container_image}}}}'

172.30.2.167 | SUCCESS | rc=0 >>
rstudio: Pulling from vcp/xrdp
a02a4930cb5d: Already exists
1082e55634b4: Already exists
7402ff065858: Already exists
7534ce324935: Already exists
3a61f4c30764: Already exists
60ec6b6b4a56: Already exists
d7f4df9e3a77: Already exists
7ba90c47242b: Pulling fs layer
2588e055f13d: Pulling fs layer
e2627e4795d8: Pulling fs layer
52151b12e1e8: Pulling fs layer
3d4b7f80ddab: Pulling fs layer
3d4b7f80ddab: Download complete
7ba90c47242b: Pull complete
2588e055f13d: Download complete
e2627e4795d8: Download complete
52151b12e1e8: Download complete
2588e055f13d: Pull complete
e2627e4795d8: Pull complete
52151b12e1e8: Pull complete
3d4b7f80ddab: Pull complete
Digest: sha256:55930d0cfe4d12da9bbac8fb25dec6f85ed413fbb9e1c8e67c04593bfff982bc
Status: Image is up to date for 192.168.2.1:5001/vcp/xrdp:rstudio



# Guacamoleにアクセスする

![Guacamoleにアクセスする](images/guacamole-002-09.png)

## ログインするのに必要となる情報

Guacamoleに登録されているユーザを確認します。

In [34]:
for user in gvars['guacamole_users']:
    print(user)

user01
user02
user03
user04
user05
user06
user07
user08
user09
user10


ハンズオン環境ではLDAPサーバに、ユーザ名と同じ値のパスワードが登録されています。Guacamoleでログインする場合は指定したユーザ名と同じ値をパスワードとして入力してください。

構築したGuacamoleにアクセスします。次のセルを実行すると表示されるURLにアクセスしてください。

In [35]:
print(f"https://{gvars['server_name']}")

https://ocs1046.vcp-handson.org


## ログイン手順

先程確認したGuacamoleのURLにアクセスするとGuacamoleのログイン画面が表示されます。
ユーザ名、パスワードを入力するとそれぞれ入力するとGuacamoleにログインできます。

> ハンズオン環境ではユーザ/パスワードに `user01` / `user01` を入力するとログインできます。

![Guacamoleログイン画面](images/guacamole-xrdp-00.png)

Guacamoleにログインするとユーザが利用できるコネクションの一覧が表示されます。

![Gucamoleコネクション一覧](images/guacamole-xrdp-01.png)

### X11環境の利用

画面の下半分に表示されている「ALL CONNECTIONS」に表示されている `rdp-user01` をクリックすると演習環境のログイン画面が表示されます。
![RDPログイン画面](images/guacamole-xrdp-02.png)

演習環境コンテナはユーザのログイン操作を契機に起動しています。タイミングによっては演習環境への接続が失敗する場合があります。接続が失敗した場合は以下のような画面が表示されます。
![コネクションエラー](images/guacamole-xrdp-05.png)

この場合、少し待って（５～１０秒程度）から「Reconnect」を選択することで演習環境のログイン画面が表示されるようになります。

演習環境のログイン画面でパスワードを入力すると演習環境にログインできます。ここで入力するパスワードはGuacamoleのログイン画面と同様で LDAP に登録されているパスワードです。

> ハンズオン環境ではユーザ名と同じ値です。

ログインに成功すると以下のような画面が表示されます。

![演習環境](images/guacamole-xrdp-03.png)

パスワードの入力ミスなどでログインに失敗すると、以下のような画面が表示されます。「OK」を選択すると再度ログイン画面が表示されますので、パスワードを入力し直してください。

![演習環境ログイン失敗](images/guacamole-xrdp-06.png)

演習環境からログアウトする場合は、上部のメニューの「システム」から「ログアウト」を選択してください。

![演習環境からのログアウト](images/guacamole-xrdp-04.png)

ログアウトを選択すると、以下のようなダイアログが表示されます。

![ログアウトダイアログ](images/guacamole-xrdp-07.png)

「ログアウト」を選択するか、既定の時間が経過すると演習環境からログアウトします。
演習環境からログアウトすると、以下のようなGuacamoleのダイアログが表示されます。

![Guacamoleログアウトダイアログ](images/guacamole-xrdp-08.png)

「Home」を選択するとGuacamoleのコネクション一覧画面に戻ります。「Reconnect」を選択すると演習環境に再度ログインを試みます。
「Logout」を選択するとGuacamoleからログアウトします。

### ターミナル環境の利用

Guacamoleのコネクション一覧表示画面で先ほどは`rdp-user01`を選択しましたが、`ssh-user01`を選択するとターミナル環境を利用することができます。
![Gucamoleコネクション一覧](images/guacamole-xrdp-01.png)

`ssh-user01` をクリックするとターミナル環境のログイン画面が表示されます。先程と同じパスワードを入力してください。

![SSHログイン画面](images/guacamole-ssh-01.png)

ログインに成功すると以下のような画面が表示されます。

![SSHログイン成功画面](images/guacamole-ssh-02.png)

コマンドラインで `exit` と入力するとターミナル環境からログアウトすることができます。

![SSHログアウト画面](images/guacamole-ssh-03.png)